<h1><center>Laboratorio 8: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla y Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias y Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Sofía Callejas
- Nombre de alumno 2: Iván Meneses


### **Link de repositorio de GitHub:** `https://github.com/ivanandresmt/lab_8.git`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [172]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = "/content/drive/MyDrive/lab_8"
except: 
    print('Ignorando conexión drive-colab')

Mounted at /content/drive


In [158]:
df_players = pd.read_csv(path +"/"+"stats_players.csv")
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [159]:
def transformar_a_binario(valor):
    if pd.isnull(valor):
        return 0
    else:
        return 1

# Aplicar la transformación a la columna "National_Position"
df_players['Seleccionado_Nacional'] = df_players['National_Position'].apply(transformar_a_binario)

# Cantidad de ejemplos para cada clase
conteo = df_players['Seleccionado_Nacional'].value_counts()
print(conteo)


0    16513
1     1075
Name: Seleccionado_Nacional, dtype: int64


Se observa que la cantidad de jugadores seleccionados es considerablemente menor (1075 casos) a la cantidad de jugadores no seleccionados (16513 casos). Esto es esperable ya que las selecciones de los países no dan abasto para admitir todos los jugadores, y sólo escogen a aquellos que desempeñan un mejor rendimiento.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

In [171]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Seleccionar columnas numéricas y categóricas
columnas_numericas = ['Height', 'Weight', 'Age', 'Weak_foot', 'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision', 'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy', 'Penalties', 'Volleys']
columnas_categoricas = ['Preffered_Foot']

# Crear ColumnTransformer con StandardScaler para características numéricas,
# y OneHotEncoder para características categóricas no ordinales
transformers = [
    ('numericas', StandardScaler(), columnas_numericas),
    ('categoricas', OneHotEncoder(drop='first', handle_unknown='ignore'), columnas_categoricas)
]

# Crear ColumnTransformer
preprocessor = ColumnTransformer(transformers)

# Definir pipelines con clasificadores
pipeline1 = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier())
])

pipeline2 = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

pipeline3 = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SVC())
])




### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


In [173]:
# Dividir los datos en características (X) y etiquetas (y)
X = df_players.drop(columns=['Name','Nationality','National_Position','Club_Position','Work_Rate'], axis=1)
y = df_players['Seleccionado_Nacional']

# Dividir los datos en conjuntos de entrenamiento y prueba, manteniendo la proporción de etiquetas
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify= y, random_state=42)

# Entrenar los Pipelines con los conjuntos de entrenamiento
pipeline1.fit(X_train, y_train)
pipeline2.fit(X_train, y_train)
pipeline3.fit(X_train, y_train)

# Evaluar los Pipelines en el conjunto de prueba
y_pred1 = pipeline1.predict(X_test)
y_pred2 = pipeline2.predict(X_test)
y_pred3 = pipeline3.predict(X_test)

# Calcular las métricas para cada clasificador
accuracy1 = accuracy_score(y_test, y_pred1)
precision1 = precision_score(y_test, y_pred1)
recall1 = recall_score(y_test, y_pred1)
f1_score1 = f1_score(y_test, y_pred1)

accuracy2 = accuracy_score(y_test, y_pred2)
precision2 = precision_score(y_test, y_pred2)
recall2 = recall_score(y_test, y_pred2)
f1_score2 = f1_score(y_test, y_pred2)

accuracy3 = accuracy_score(y_test, y_pred3)
precision3 = precision_score(y_test, y_pred3)
recall3 = recall_score(y_test, y_pred3)
f1_score3 = f1_score(y_test, y_pred3)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [174]:
# Imprimir métricas
print("Métricas del clasificador 1: DecisionTreeClassifier") 
print("Accuracy:", accuracy1)
print("Precision:", precision1)
print("Recall:", recall1)
print("F1_Score:", f1_score1)

print("")

print("Métricas del clasificador 2: RandomForestClassifier") 
print("Accuracy:", accuracy2)
print("Precision:", precision2)
print("Recall:", recall2)
print("F1_Score:", f1_score2)

print("")

print("Métricas del clasificador 3: SVC") 
print("Accuracy:", accuracy3)
print("Precision:", precision3)
print("Recall:", recall3)
print("F1_Score:", f1_score3)

Métricas del clasificador 1: DecisionTreeClassifier
Accuracy: 0.8982376350198976
Precision: 0.2154255319148936
Recall: 0.25077399380804954
F1_Score: 0.23175965665236054

Métricas del clasificador 2: RandomForestClassifier
Accuracy: 0.9412545006632557
Precision: 0.6666666666666666
Recall: 0.0804953560371517
F1_Score: 0.143646408839779

Métricas del clasificador 3: SVC
Accuracy: 0.9387909797233277
Precision: 0.0
Recall: 0.0
F1_Score: 0.0


 

1.   ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva(la que indica que un jugador es seleccionado nacional)?:
      
      Accuracy implica el porcentaje de casos en que el modelo ha acertado si un jugador es seleccionado nacional. Por otro lado Precision indica el porcentaje de seleccionados que acierta del total predicho como seleccionado, y por último recall hace referencia al porcentaje de jugadores seleccionados que es capaz de predecir el modelo considerando el total de seleccionados.

2.   ¿Podrían mejorarse los resultados?, ¿Cómo?
     Sí, podría mejorarse mediante un preprocesamiento (columntransform) más acabado de los datos que permita adaptarse mejor a cada una de las columnas en cuestion. 

3.   ¿Influye la cantidad de ejemplos por clase?
     Sí, ya que dependiendo de la cantidad de elementos por clase se establece una proporción que dictamina la probabilidad de exito de nuestro modelo.






## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [165]:
def transformar_posicion_jugador(valor):    
    if valor in ['ST', 'CF', 'RS', 'LS', 'LF', 'RF']:    
      return "ataque"
    
    elif valor in  ['RW', 'CAM', 'LW', 'LCM', 'RCM', 'LAM', 'RAM']: 
      return "central_ataque"
    
    elif valor in ['RM', 'CM', 'LM']:
      return "central"
    
    elif valor in ['RWB', 'CDM', 'LWB', 'RCB', 'LCB']:    
      return "central_defensa"
    
    elif valor in ['RB', 'CB', 'LB', 'LDM', 'RDM']:    
      return "defensa"
    
    elif valor in ['GK']:       
      return "arquero"
    
    else:
      return valor

# Aplicar la transformación a la columna "Club Position"
df_players['Posicion_Jugador'] = df_players['Club_Position'].apply(transformar_posicion_jugador)

# Cantidad de ejemplos para cada clase
conteo = df_players['Posicion_Jugador'].value_counts()
print(conteo)

Sub                7492
Res                3146
defensa            1712
central_defensa    1473
central_ataque     1360
central             907
ataque              865
arquero             632
Name: Posicion_Jugador, dtype: int64


In [167]:
from sklearn.linear_model import LogisticRegression

df_player_without_na = df_players.dropna(inplace=False)

# Dividir los datos en características (X) y etiquetas (y)
X = df_player_without_na.drop(columns=['Name','National_Position','Club_Position','Work_Rate'], axis=1)
y = df_player_without_na['Posicion_Jugador']

# Seleccionar columnas numéricas y categóricas
columnas_numericas = ['Height',
 'Weight',
 'Age',
 'Weak_foot',
 'Skill_Moves',
 'Ball_Control',
 'Dribbling',
 'Marking',
 'Sliding_Tackle',
 'Standing_Tackle',
 'Aggression',
 'Reactions',
 'Interceptions',
 'Vision',
 'Composure',
 'Crossing',
 'Short_Pass',
 'Long_Pass',
 'Acceleration',
 'Speed',
 'Stamina',
 'Strength',
 'Balance',
 'Agility',
 'Jumping',
 'Heading',
 'Shot_Power',
 'Finishing',
 'Long_Shots',
 'Curve',
 'Freekick_Accuracy',
 'Penalties',
 'Volleys',
 'Seleccionado_Nacional']
columnas_categoricas = [
 'Nationality',
 'Preffered_Foot']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Definir el ColumnTransformer con OneHotEncoder y StandardScaler
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), columnas_categoricas),
        ('scale', StandardScaler(), columnas_numericas)
    ])

# Definir el Pipeline con el preprocesamiento y el clasificador
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(multi_class='multinomial', solver='lbfgs'))
])

# Entrenar el Pipeline con el conjunto de entrenamiento
pipeline.fit(X_train, y_train)

# Evaluar el Pipeline en el conjunto de prueba
y_pred = pipeline.predict(X_test)

# Calcular las métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

# Mostrar las métricas
print("Métricas:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Métricas:
Accuracy: 0.4674922600619195
Precision: 0.4505489722881027
Recall: 0.4834042849969634
F1 Score: 0.46288474154005677


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


**Respuesta**:

```
Se obtiene una precision del 45% que significa que de todos los valores predichos se ha acertado en un 45% de las veces. Además, se obtiene un recall de 48% que significa que del total de posiciones de los jugadores, se logra acertar en un 48% de las veces (del universo total).
```

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

R2 es una métrica de regresión que indica qué tan bien un modelo se ajusta a los datos reales. Varía entre 0 y 1, donde 1 significa un ajuste perfecto y 0 significa un ajuste deficiente. R2 representa la proporción de la variabilidad de la variable dependiente explicada por el modelo.

In [168]:
#df_players = pd.read_csv(path +"/"+"stats_players.csv")
df_sueldos = pd.read_csv(path + "/"+ "salarios.csv")
df_sueldos.head()

,Unnamed: 0,Player,Weekly Salary
0,1528,Lionel Messi,1538905.0
1,1006,Cristiano Ronaldo,1248536.0
2,1529,Antoine Griezmann,997032.0
3,2004,Neymar,797726.0
4,1530,Gareth Bale,607706.0


In [96]:
df_regresion = df_players.merge(df_sueldos, left_on = "Name" , right_on = "Player")
df_regresion.head()

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Seleccionado_Nacional,Posicion_Jugador,Unnamed: 0,Player,Weekly Salary
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,81,76,85,88,1,central_ataque,1006,Cristiano Ronaldo,1248536.0
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,88,89,90,74,85,1,central_ataque,1528,Lionel Messi,1538905.0
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,77,79,84,81,83,1,central_ataque,2004,Neymar,797726.0
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,86,84,85,88,1,ataque,1531,Luis Suárez,508923.0
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,16,14,11,47,11,1,arquero,498,Manuel Neuer,326233.0


In [169]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Seleccionar columnas numéricas y categóricas
columnas_numericas = ['Height',
 'Weight',
 'Age',
 'Weak_foot',
 'Skill_Moves',
 'Ball_Control',
 'Dribbling',
 'Marking',
 'Sliding_Tackle',
 'Standing_Tackle',
 'Aggression',
 'Reactions',
 'Interceptions',
 'Vision',
 'Composure',
 'Crossing',
 'Short_Pass',
 'Long_Pass',
 'Acceleration',
 'Speed',
 'Stamina',
 'Strength',
 'Balance',
 'Agility',
 'Jumping',
 'Heading',
 'Shot_Power',
 'Finishing',
 'Long_Shots',
 'Curve',
 'Freekick_Accuracy',
 'Penalties',
 'Volleys']
columnas_categoricas = ['Posicion_Jugador',
 'Seleccionado_Nacional']

# Definir ColumnTransformer con StandardScaler para características numéricas y OneHotEncoder para características categóricas
transformer = ColumnTransformer([
    ('numericas', StandardScaler(), columnas_numericas),
    ('categoricas', OneHotEncoder(drop='first'), columnas_categoricas)
])

# Definir pipeline con el ColumnTransformer y el regresor
pipeline = Pipeline([
    ('preprocessor', transformer),
    ('regressor', RandomForestRegressor())
])

# Dividir los datos en características (X) y etiquetas (y)
X = df_regresion[columnas_numericas + columnas_categoricas]
y = df_regresion['Weekly Salary']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Entrenar el pipeline con los conjuntos de entrenamiento
pipeline.fit(X_train, y_train)

# Predecir con el pipeline en el conjunto de prueba
y_pred = pipeline.predict(X_test)

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(y_test, y_pred)


In [170]:
print(r2)

0.41269876689076945


**Respuesta**:

```
La regresión entrega un R^2 = 0.413 lo que es suficientemente bueno para nuestro interés, sin embargo, es posible mejorar esta métrica mediante la correcta selección de variables independientes (regresores) que explicarían mejor el salario de los jugadores, y además también se recomienda agregar variables (regresores) que sean más reveladores para predecir el salario de un jugador.
```

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>